In [1]:
from open_broker_report import open_broker_report
file_dir=r'C:\Users\Kart\jupyter_projects\web\broker_info'

In [2]:
file_dir

'C:\\Users\\Kart\\jupyter_projects\\web\\broker_info'

In [6]:
ob=open_broker_report(file_dir,"_173364i_")
#ob._extract_title_info()
#ob._extract_section_definition(True,True) 
#ob._extract_unified_account_totally()
#ob._extract_unified_guarantee_totally()

#ob._extract_unified_preclosing_assets()
#ob._extract_unified_closing_assets()
#ob._extract_spot_main_deals_conclusion()
#ob._extract_spot_main_deals_executed()
#ob._extract_unified_non_trade_money_operations()

#ob._extract_unified_spot_non_trade_security_operations()
#ob._extract_spot_portfolio_security_params()

In [9]:
ob._extract_unified_account_totally()

{'income_balance_fact': {'RUB': {'value': 0.0},
  'text_description': 'Входящий остаток (факт)'},
 'broker_fee_for_info': {'RUB': {'value': -1790.0},
  'text_description': 'Вознаграждение Брокера за организацию доступа к биржевым торгам с предоставлением информации клиентам, необходимой для совершения операций и сделок'},
 'cashflow': {'RUB': {'value': 2091410.69},
  'USD': {'value': 346.08},
  'text_description': 'Движение средств'},
 'broker_repo_comission': {'RUB': {'value': -11.24},
  'text_description': 'Комиссия Брокера за заключение Специальных сделок РЕПО'},
 'broker_fee_for_deals': {'RUB': {'value': -1651.74},
  'text_description': 'Комиссия Брокера/Доп. комиссия Брокера "Сборы ТС" за заключение сделок'},
 'broker_loan_securities_comission': {'RUB': {'value': -0.98},
  'text_description': 'Комиссия за сделки займа ЦБ на условиях Примерных условий сделок займа ценных бумаг'},
 'ncd': {'RUB': {'value': -3800.12}, 'text_description': 'НКД'},
 'repayment_ncd': {'RUB': {'value': 17

In [21]:
a=ob.json_report()

In [23]:
a['title_info']['period']

('2018-12-03', '2021-06-22')

In [17]:
b=[el for el in a if el['comment']=='Поставлены на торги средства клиента  173364']

In [18]:
b

[{'operation_date': '2018-12-24T00:00:00',
  'currency_code': 'RUB',
  'amount': 100000.0,
  'comment': 'Поставлены на торги средства клиента  173364'},
 {'operation_date': '2019-03-11T00:00:00',
  'currency_code': 'RUB',
  'amount': 120000.0,
  'comment': 'Поставлены на торги средства клиента  173364'},
 {'operation_date': '2019-04-09T00:00:00',
  'currency_code': 'RUB',
  'amount': 200000.0,
  'comment': 'Поставлены на торги средства клиента  173364'},
 {'operation_date': '2019-07-08T00:00:00',
  'currency_code': 'RUB',
  'amount': 200000.0,
  'comment': 'Поставлены на торги средства клиента  173364'},
 {'operation_date': '2019-09-09T00:00:00',
  'currency_code': 'RUB',
  'amount': 100000.0,
  'comment': 'Поставлены на торги средства клиента  173364'},
 {'operation_date': '2019-10-14T00:00:00',
  'currency_code': 'RUB',
  'amount': 100000.0,
  'comment': 'Поставлены на торги средства клиента  173364'},
 {'operation_date': '2019-12-23T00:00:00',
  'currency_code': 'RUB',
  'amount': 1

In [5]:
#ob._extract_title_info(print_raw=True)
#ob._extract_section_definition(True,False) 
ob._extract_unified_account_totally(print_raw=True)
#ob._extract_unified_guarantee_totally(print_raw=True)

#ob._extract_unified_preclosing_assets(print_raw=True)
#ob._extract_unified_closing_assets(print_raw=True)
#ob._extract_spot_main_deals_conclusion(print_raw=True)
#ob._extract_spot_main_deals_executed(print_raw=True)
#ob._extract_unified_non_trade_money_operations(print_raw=True)

#ob._extract_unified_spot_non_trade_security_operations(print_raw=True)
#ob._extract_spot_portfolio_security_params(print_raw=True)

AttributeError: 'NoneType' object has no attribute 'getchildren'

In [ ]:
    def __json_report(self,report=False):
        res={}
        res['title_info']=self._extract_title_info()
        res['section_definition']=self._extract_section_definition(True,True) 
        
        res['spot_main_deals_conclusion']=self._extract_spot_main_deals_conclusion()
        res['spot_main_deals_executed']=self._extract_spot_main_deals_executed()

        res['spot_portfolio_security_params']=self._extract_spot_portfolio_security_params()
        
        if res['title_info']['agreement']['dgvr']=='Договор на ведение индивидуального инвестиционного счета: ':
            res['account_totally']=self._extract_spot_account_totally()
            res['unified_closing_assets']=self._extract_spot_preclosing_assets()
            res['closing_assets']=self._extract_spot_assets()       
            res['non_trade_money_operations']=self._extract_spot_non_trade_money_operations()       
            return res

        res['account_totally']=self._extract_unified_account_totally()
        res['guarantee_totally']=self._extract_unified_guarantee_totally()
        res['non_trade_money_operations']=self._extract_unified_non_trade_money_operations()        
        res['preclosing_assets']=self._extract_unified_preclosing_assets()
        res['closing_assets']=self._extract_unified_closing_assets()        
        res['spot_non_trade_security_operations']=self._extract_unified_spot_non_trade_security_operations()

        return res
    
    def json_report(self,full_report:bool=False):
        report=self.__json_report()
        if not full_report:
            small_report={'period':report['title_info']['period'],
                          'assets_cb_value':report["account_totally"]["assets_cb_value_fact"],
                          
                
                            }
            
            pass
        return report

In [27]:
a["account_totally"]["assets_cb_value_fact"]

{'RUB': {'value': 2886710.67},
 'text_description': 'Оценка активов, по курсу ЦБ (факт)'}